In [1]:
import pandas as pd

We read and compute the efficiences for each volume and isotope (214Bi, 208Tl)

In [2]:
efficiencies = pd.read_csv('efficiencies_2.4_2.5_MeV.csv')
efficiencies['eff'] = efficiencies.nsaved / efficiencies.nsim
efficiencies.drop(['nsim', 'nsaved', 'npass', 'nenergy'], axis = 1, inplace=True)

We read the activities now and set up the DF to merge with the efficiencies DF.

In [3]:
activities = pd.read_csv('activities_NEXT100.csv', sep=';', decimal = ',')
activities = activities.rename(columns = {'NEXUS VOLUME':'G4Volume', 'Bi-214':'214Bi', 'Co-60':'60Co', 'K-40':'40K', 'Tl-208':'208Tl'})

Bi_act = activities[['G4Volume', '214Bi']].copy()
Tl_act = activities[['G4Volume', '208Tl']].copy()
Bi_act['Isotope'] = '214Bi'
Tl_act['Isotope'] = '208Tl'

act = Bi_act.rename(columns={'214Bi':'act'}).append(Tl_act.rename(columns={'208Tl':'act'}))

eff_and_act = efficiencies.merge(act, on = ['G4Volume', 'Isotope'], how = 'outer')


We add a column with the counts/year we want to have (using the ones in the 2.4-2.5MeV region from Gonzalo's thesis)

In [4]:
eff_and_act['final_counts'] = eff_and_act['Isotope'].apply(lambda x: {'208Tl':1800, 
                                                                      '214Bi':296, 
                                                                      '0nubb':0, 
                                                                      'muons':900, 
                                                                      '137Xe':33}[x])

In [5]:
factor = 100

#for activities in mBq
year = (3600 * 24 * 365) / 1000

eff_and_act['needed_counts'] = (eff_and_act.final_counts / eff_and_act.eff) * factor
eff_and_act['exposure'] = eff_and_act.needed_counts / (eff_and_act.act * year)


In [6]:
eff_and_act

,Isotope,G4Volume,eff,act,final_counts,needed_counts,exposure
0,214Bi,DB_PLUG,1.000400e-07,155.000,296,2.958817e+11,6.053127e+04
1,214Bi,EP_COPPER_PLATE,3.802124e-06,313.000,296,7.785123e+09,7.887049e+02
2,214Bi,GATE_RING,3.953236e-05,4.760,296,7.487537e+08,4.987989e+03
3,214Bi,HDPE_TUBE,4.443023e-05,3.070,296,6.662131e+08,6.881264e+03
4,214Bi,VESSEL,1.520359e-07,3420.000,296,1.946909e+11,1.805148e+03
5,214Bi,PEDESTAL,2.500000e-08,358.000,296,1.184000e+12,1.048726e+05
6,214Bi,SHIELDING_LEAD,2.936508e-08,5320.000,296,1.008000e+12,6.008171e+03
7,214Bi,ANODE_RING,4.014916e-05,4.760,296,7.372508e+08,4.911359e+03
8,214Bi,SIPM_BOARD,8.291265e-05,21.800,296,3.570022e+08,5.192874e+02
9,214Bi,OPTICAL_PAD,2.528092e-05,7.920,296,1.170844e+09,4.687779e+03
